# Analisis optimizaciones fichero .Parquet

Para realizar el analisis, después de arrancar la sesión de Spark hay que ir ejecutando el codigo de las celdas y posteriormente meterse en el Spark UI para comparar los jobs ejecutados. Asi se puede observar las diferencias entre las consultas al fichero .csv y el .parquet.

In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.3`


import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window



//Reducir numero logs
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)


//For adding extra dependenies
import $ivy.`org.typelevel::cats-core:1.6.0`

//Plotly
import $ivy.`org.plotly-scala::plotly-almond:0.8.3`
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._


//Iniciamos session de Spark
val spark = {
  NotebookSparkSession.builder()
     .master("local[*]")
    //.config("spark.executor.instances", "4") // Número de ejecutores
    //.config("spark.executor.memory", "4g") // Memoria por ejecutor
    //.config("spark.executor.cores", "2") // Núcleos por ejecutor
    //.config("spark.driver.memory", "4g") // Memoria del driver
    //.config("spark.sql.shuffle.partitions", "8") // Número de particiones para operaciones de shuffle
    .getOrCreate()
}
import spark.implicits._

Loading spark-stubs

Creating SparkSession

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

import $ivy.$
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window
import org.apache.log4j.{Level, Logger}
import $ivy.$
import $ivy.$
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._


//Iniciamos session de Spark

spark: SparkSession = org.apache.spark.sql.SparkSession@1a099168
import spark.implicits._

#### Lectura simple de datos

In [59]:
val pathCsv = "../../data/csv/dsBalanceNacionalAnalisis.csv"      
val pathParquet = "../../data/parquet/dsBalanceNacionalParticionado.parquet"

// Medir tiempo de lectura CSV
val startCsv = System.nanoTime()
val dfFromCsv: DataFrame = spark.read
    .option("header", "true") // Para que Spark lea la primera linea como encabezado
    .option("inferSchema", "true") // Para inferir automáticamente el esquema
    .csv(pathCsv)
val endCsv = System.nanoTime()
println(s"Tiempo de lectura CSV: ${(endCsv - startCsv) / 1e9} segundos")


// Medir tiempo de lectura Parquet
val startParquet = System.nanoTime()
val dfFromParquet: DataFrame = spark.read.parquet(pathParquet)
val endParquet = System.nanoTime()
println(s"Tiempo de lectura Parquet: ${(endParquet - startParquet) / 1e9} segundos")

// Ver esquema
dfFromCsv.explain()
dfFromParquet.explain()

Tiempo de lectura CSV: 0.3589797 segundos


Tiempo de lectura Parquet: 0.1673615 segundos
== Physical Plan ==
*(1) FileScan csv [Familia#1635,Tipo#1636,Compuesto#1637,Porcentaje#1638,Valor#1639,Fecha#1640,BajasEmisiones#1641,Año#1642,Mes#1643] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/C:/Proyectos/spark-datos-energia/data/csv/dsBalanceNacionalAnalisis.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Familia:string,Tipo:string,Compuesto:boolean,Porcentaje:double,Valor:double,Fecha:timestam...
== Physical Plan ==
*(1) FileScan parquet [Familia#1653,Tipo#1654,Compuesto#1655,Porcentaje#1656,Valor#1657,Fecha#1658,BajasEmisiones#1659,Año#1660,Mes#1661] Batched: true, Format: Parquet, Location: InMemoryFileIndex[file:/C:/Proyectos/spark-datos-energia/data/parquet/dsBalanceNacionalParticiona..., PartitionCount: 169, PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Familia:string,Tipo:string,Compuesto:boolean,Porcentaje:double,Valor:double,Fecha:timestam...


pathCsv: String = "../../data/csv/dsBalanceNacionalAnalisis.csv"
pathParquet: String = "../../data/parquet/dsBalanceNacionalParticionado.parquet"
startCsv: Long = 783512048687500L
dfFromCsv: DataFrame = [Familia: string, Tipo: string ... 7 more fields]
endCsv: Long = 783512407667200L
startParquet: Long = 783512407933500L
dfFromParquet: DataFrame = [Familia: string, Tipo: string ... 7 more fields]
endParquet: Long = 783512575295000L

#### Lectura parcial (solo algunas columnas)

In [35]:
val columnasFromCsv = dfFromCsv
    .select("Fecha", "Tipo", "Valor")

columnasFromCsv.explain()

val columnasFromParquet = dfFromParquet
    .select("Fecha", "Tipo", "Valor")

columnasFromParquet.explain()

== Physical Plan ==
*(1) Project [Fecha#425, Tipo#421, Valor#424]
+- *(1) FileScan csv [Tipo#421,Valor#424,Fecha#425] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/C:/Proyectos/spark-datos-energia/data/csv/dsBalanceNacional11-25.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Tipo:string,Valor:double,Fecha:timestamp>
== Physical Plan ==
*(1) Project [Fecha#439, Tipo#435, Valor#438]
+- *(1) FileScan parquet [Tipo#435,Valor#438,Fecha#439] Batched: true, Format: Parquet, Location: InMemoryFileIndex[file:/C:/Proyectos/spark-datos-energia/data/parquet/dsBalanceNacional11-25.parq..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Tipo:string,Valor:double,Fecha:timestamp>


columnasFromCsv: DataFrame = [Fecha: timestamp, Tipo: string ... 1 more field]
columnasFromParquet: DataFrame = [Fecha: timestamp, Tipo: string ... 1 more field]

#### Lectura filtrada

In [63]:
val pathCsv = "../../data/csv/dsBalanceNacionalAnalisis.csv"      
val pathParquet = "../../data/parquet/dsBalanceNacionalParticionado.parquet"

//csv
val startCsv = System.nanoTime()
val dfCsvFiltered = spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv(pathCsv)
    .filter($"Año" === 2024 && $"Mes" === 1)// Filtro directo en la columna sin transformaciones

val endCsv = System.nanoTime()

println(s"Tiempo de lectura CSV: ${(endCsv - startCsv) / 1e9} segundos")

dfCsvFiltered.explain() // Ver plan de ejecución
dfCsvFiltered.show(1)

Tiempo de lectura CSV: 0.3754405 segundos
== Physical Plan ==
*(1) Project [Familia#1801, Tipo#1802, Compuesto#1803, Porcentaje#1804, Valor#1805, Fecha#1806, BajasEmisiones#1807, Año#1808, Mes#1809]
+- *(1) Filter (((isnotnull(Año#1808) && isnotnull(Mes#1809)) && (Año#1808 = 2024)) && (Mes#1809 = 1))
   +- *(1) FileScan csv [Familia#1801,Tipo#1802,Compuesto#1803,Porcentaje#1804,Valor#1805,Fecha#1806,BajasEmisiones#1807,Año#1808,Mes#1809] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/C:/Proyectos/spark-datos-energia/data/csv/dsBalanceNacionalAnalisis.csv], PartitionFilters: [], PushedFilters: [IsNotNull(Año), IsNotNull(Mes), EqualTo(Año,2024), EqualTo(Mes,1)], ReadSchema: struct<Familia:string,Tipo:string,Compuesto:boolean,Porcentaje:double,Valor:double,Fecha:timestam...


+---------+----------+---------+-------------------+---------+-------------------+--------------+----+---+
|  Familia|      Tipo|Compuesto|         Porcentaje|    Valor|              Fecha|BajasEmisiones| Año|Mes|
+---------+----------+---------+-------------------+---------+-------------------+--------------+----+---+
|Renovable|Hidráulica|    false|0.19718853720380722|58381.148|2024-01-01 01:00:00|          true|2024|  1|
+---------+----------+---------+-------------------+---------+-------------------+--------------+----+---+
only showing top 1 row



pathCsv: String = "../../data/csv/dsBalanceNacionalAnalisis.csv"
pathParquet: String = "../../data/parquet/dsBalanceNacionalParticionado.parquet"
startCsv: Long = 784023217208500L
dfCsvFiltered: Dataset[Row] = [Familia: string, Tipo: string ... 7 more fields]
endCsv: Long = 784023592649000L

In [58]:
//Parquet
val startParquet = System.nanoTime()

val dfParquetFiltered = spark.read
    .parquet(pathParquet)
    .filter($"Año" === 2024 && $"Mes" === 1) // Filtro directo en la columna sin transformaciones

val endParquet = System.nanoTime()

println(s"Tiempo de lectura Parquet: ${(endParquet - startParquet) / 1e9} segundos")

dfParquetFiltered.explain() // Ver plan de ejecución
dfParquetFiltered.show(1)

Tiempo de lectura Parquet: 0.2106526 segundos
== Physical Plan ==
*(1) FileScan parquet [Familia#1578,Tipo#1579,Compuesto#1580,Porcentaje#1581,Valor#1582,Fecha#1583,BajasEmisiones#1584,Año#1585,Mes#1586] Batched: true, Format: Parquet, Location: InMemoryFileIndex[file:/C:/Proyectos/spark-datos-energia/data/parquet/dsBalanceNacionalParticiona..., PartitionCount: 1, PartitionFilters: [isnotnull(Año#1585), isnotnull(Mes#1586), (Año#1585 = 2024), (Mes#1586 = 1)], PushedFilters: [], ReadSchema: struct<Familia:string,Tipo:string,Compuesto:boolean,Porcentaje:double,Valor:double,Fecha:timestam...


+---------+----------+---------+-------------------+---------+-------------------+--------------+----+---+
|  Familia|      Tipo|Compuesto|         Porcentaje|    Valor|              Fecha|BajasEmisiones| Año|Mes|
+---------+----------+---------+-------------------+---------+-------------------+--------------+----+---+
|Renovable|Hidráulica|    false|0.19718853720380722|58381.148|2024-01-01 01:00:00|          true|2024|  1|
+---------+----------+---------+-------------------+---------+-------------------+--------------+----+---+
only showing top 1 row



startParquet: Long = 783399512523100L
dfParquetFiltered: Dataset[Row] = [Familia: string, Tipo: string ... 7 more fields]
endParquet: Long = 783399723175700L

#### Agrupaciones

In [65]:
val startCsv = System.nanoTime()

val dfCsv = spark.read
    .option("header", "true")
    .option("inferSchema", "true") // Spark debe inferir el esquema en cada lectura
    .csv(pathCsv)
    .filter($"Año" === 2024 && $"Mes" === 1) // NO optimizado (lee todo el archivo)
    .filter($"Familia" === "Renovable") // NO optimizado (lee todas las filas)
    .groupBy("Tipo")
    .agg(
        sum("Valor").as("TotalGenerado"),
        avg("Porcentaje").as("PorcentajePromedio")
    )

val endCsv = System.nanoTime()
println(s"Tiempo de lectura CSV: ${(endCsv - startCsv) / 1e9} segundos")

dfCsv.explain(true)
dfCsv.show(5)

Tiempo de lectura CSV: 0.4236098 segundos
== Parsed Logical Plan ==
'Aggregate [Tipo#1868], [Tipo#1868, sum('Valor) AS TotalGenerado#1895, avg('Porcentaje) AS PorcentajePromedio#1897]
+- Filter (Familia#1867 = Renovable)
   +- Filter ((Año#1874 = 2024) && (Mes#1875 = 1))
      +- Relation[Familia#1867,Tipo#1868,Compuesto#1869,Porcentaje#1870,Valor#1871,Fecha#1872,BajasEmisiones#1873,Año#1874,Mes#1875] csv

== Analyzed Logical Plan ==
Tipo: string, TotalGenerado: double, PorcentajePromedio: double
Aggregate [Tipo#1868], [Tipo#1868, sum(Valor#1871) AS TotalGenerado#1895, avg(Porcentaje#1870) AS PorcentajePromedio#1897]
+- Filter (Familia#1867 = Renovable)
   +- Filter ((Año#1874 = 2024) && (Mes#1875 = 1))
      +- Relation[Familia#1867,Tipo#1868,Compuesto#1869,Porcentaje#1870,Valor#1871,Fecha#1872,BajasEmisiones#1873,Año#1874,Mes#1875] csv

== Optimized Logical Plan ==
Aggregate [Tipo#1868], [Tipo#1868, sum(Valor#1871) AS TotalGenerado#1895, avg(Porcentaje#1870) AS PorcentajePromedio#189

+-------------------+------------------+--------------------+
|               Tipo|     TotalGenerado|  PorcentajePromedio|
+-------------------+------------------+--------------------+
|Residuos renovables| 69243.12599999999|0.005950484155910...|
| Solar fotovoltaica|1932458.7750000001| 0.16709750936029064|
|      Solar térmica| 94242.96699999998|0.008281224136326733|
|   Otras renovables| 283523.7090000001| 0.02439302139040502|
|        Hidroeólica| 739.2880000000001|6.366359586924555E-5|
+-------------------+------------------+--------------------+
only showing top 5 rows



startCsv: Long = 785003079486700L
dfCsv: DataFrame = [Tipo: string, TotalGenerado: double ... 1 more field]
endCsv: Long = 785003503096500L

In [67]:
//Parquet
val startParquet = System.nanoTime()

val dfParquet = spark.read
    .parquet(pathParquet)
    .filter($"Año" === 2024 && $"Mes" === 1) // Partition Pruning (lee solo las carpetas necesarias)
    .filter($"Familia" === "Renovable") // Predicate Pushdown (lee solo las filas necesarias)
    .groupBy("Tipo")
    .agg(
        sum("Valor").as("TotalGenerado"),
        avg("Porcentaje").as("PorcentajePromedio")
    )
val endParquet = System.nanoTime()

println(s"Tiempo de lectura Parquet: ${(endParquet - startParquet) / 1e9} segundos")


dfParquet.explain(true)
dfParquet.show(5)

Tiempo de lectura Parquet: 0.2106089 segundos
== Parsed Logical Plan ==
'Aggregate [Tipo#1931], [Tipo#1931, sum('Valor) AS TotalGenerado#1958, avg('Porcentaje) AS PorcentajePromedio#1960]
+- Filter (Familia#1930 = Renovable)
   +- Filter ((Año#1937 = 2024) && (Mes#1938 = 1))
      +- Relation[Familia#1930,Tipo#1931,Compuesto#1932,Porcentaje#1933,Valor#1934,Fecha#1935,BajasEmisiones#1936,Año#1937,Mes#1938] parquet

== Analyzed Logical Plan ==
Tipo: string, TotalGenerado: double, PorcentajePromedio: double
Aggregate [Tipo#1931], [Tipo#1931, sum(Valor#1934) AS TotalGenerado#1958, avg(Porcentaje#1933) AS PorcentajePromedio#1960]
+- Filter (Familia#1930 = Renovable)
   +- Filter ((Año#1937 = 2024) && (Mes#1938 = 1))
      +- Relation[Familia#1930,Tipo#1931,Compuesto#1932,Porcentaje#1933,Valor#1934,Fecha#1935,BajasEmisiones#1936,Año#1937,Mes#1938] parquet

== Optimized Logical Plan ==
Aggregate [Tipo#1931], [Tipo#1931, sum(Valor#1934) AS TotalGenerado#1958, avg(Porcentaje#1933) AS Porcentaje

+-------------------+------------------+--------------------+
|               Tipo|     TotalGenerado|  PorcentajePromedio|
+-------------------+------------------+--------------------+
|Residuos renovables| 69243.12599999999|0.005950484155910...|
| Solar fotovoltaica|1932458.7750000001| 0.16709750936029064|
|      Solar térmica| 94242.96699999998|0.008281224136326733|
|   Otras renovables| 283523.7090000001| 0.02439302139040502|
|        Hidroeólica| 739.2880000000001|6.366359586924555E-5|
+-------------------+------------------+--------------------+
only showing top 5 rows



startParquet: Long = 785067117229600L
dfParquet: DataFrame = [Tipo: string, TotalGenerado: double ... 1 more field]
endParquet: Long = 785067327838500L

In [18]:
dfFromParquet.groupBy("Tipo").count().show()

+--------------------+-----+
|                Tipo|count|
+--------------------+-----+
| Residuos renovables| 5145|
|     Demanda en b.c.| 5145|
|  Turbinación bombeo| 5145|
|Saldo almacenamiento| 5145|
|     Consumos bombeo| 5145|
|  Solar fotovoltaica| 5145|
|       Solar térmica| 5141|
|Generación no ren...| 5145|
|    Otras renovables| 5145|
|             Nuclear| 5145|
|         Hidroeólica| 3838|
|      Turbina de gas| 5145|
|Generación renovable| 5145|
|Residuos no renov...| 5145|
|        Cogeneración| 5145|
|Saldo I. internac...| 5145|
|       Carga batería|  727|
|     Entrega batería|  665|
|      Motores diésel| 5145|
|              Eólica| 5145|
+--------------------+-----+
only showing top 20 rows



In [15]:
//parquet

val startParquet = System.nanoTime()
val ventanaParquet = Window.partitionBy("Año")

val datosBalanceAnualesParquet = dfFromParquet
    .withColumn("Año", date_format($"Fecha", "yyyy"))
    .groupBy($"Año", $"Familia", $"Tipo", $"Compuesto", $"BajasEmisiones")
    .agg(
        sum("Valor").as("ValorAnual"),
        avg("Porcentaje").as("PorcentajeAnualSobreFamilia")
    )
    .withColumn(
        "TotalGenerado", 
        round(
            sum(
                when(!$"Compuesto" && $"Familia" =!= "Demanda" ,$"ValorAnual")
                    .otherwise(0)).over(ventanaParquet),2)
    )
    .withColumn(
        "PorcentajeSobreTotal", 
        round($"ValorAnual" / $"TotalGenerado" * 100,2))
    .withColumn(
        "PorcentajeBajasEmisiones", 
        round(
            sum(
                when($"BajasEmisiones" && !$"Compuesto" && $"Familia" =!= "Demanda", $"ValorAnual")
                    .otherwise(0)).over(ventanaParquet) / $"TotalGenerado" * 100,2)
    )

// Mostrar los primeros resultados y explicar el plan de ejecución
datosBalanceAnualesParquet.show(5)
//datosBalanceAnualesParquet.explain(true)
val endParquet = System.nanoTime()
println(s"Tiempo ejecución Parquet: ${(endParquet - startParquet) / 1e9} segundos")

+----+------------+--------------------+---------+--------------+-------------------+---------------------------+--------------+--------------------+------------------------+
| Año|     Familia|                Tipo|Compuesto|BajasEmisiones|         ValorAnual|PorcentajeAnualSobreFamilia| TotalGenerado|PorcentajeSobreTotal|PorcentajeBajasEmisiones|
+----+------------+--------------------+---------+--------------+-------------------+---------------------------+--------------+--------------------+------------------------+
|2016|No-Renovable|    Turbina de vapor|    false|         false| 2536143.0030000005|         0.0167077641039327|2.5700810617E8|                0.99|                   61.13|
|2016|No-Renovable|      Turbina de gas|    false|         false|  616037.3019999998|       0.003893066689377...|2.5700810617E8|                0.24|                   61.13|
|2016|   Renovable|       Solar térmica|    false|          true|  5071201.701999999|       0.054412720483706205|2.5700810617

startParquet: Long = 704686483035900L
ventanaParquet: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@52c3df61
datosBalanceAnualesParquet: DataFrame = [Año: string, Familia: string ... 8 more fields]
endParquet: Long = 704687162083400L

In [14]:
//csv
val startCsv = System.nanoTime()

val ventanaCsv = Window.partitionBy("Año")
val datosBalanceAnualesCsv = dfFromCsv
    .withColumn("Año", date_format($"Fecha", "yyyy"))
    .groupBy($"Año", $"Familia", $"Tipo", $"Compuesto", $"BajasEmisiones")
    .agg(
        sum("Valor").as("ValorAnual"),
        avg("Porcentaje").as("PorcentajeAnualSobreFamilia")
    )
    .withColumn(
        "TotalGenerado", 
        round(
            sum(
                when(!$"Compuesto" && $"Familia" =!= "Demanda" ,$"ValorAnual")
                    .otherwise(0)).over(ventanaCsv),2)
    )
    .withColumn(
        "PorcentajeSobreTotal", 
        round($"ValorAnual" / $"TotalGenerado" * 100,2))
    .withColumn(
        "PorcentajeBajasEmisiones", 
        round(
            sum(
                when($"BajasEmisiones" && !$"Compuesto" && $"Familia" =!= "Demanda", $"ValorAnual")
                    .otherwise(0)).over(ventanaCsv) / $"TotalGenerado" * 100,2)
    )

// Mostrar los primeros resultados y explicar el plan de ejecución
datosBalanceAnualesCsv.show(5)
//datosBalanceAnualesCsv.explain(true)

val endCsv = System.nanoTime()
println(s"Tiempo ejecución CSV: ${(endCsv - startCsv) / 1e9} segundos")

+----+------------+--------------------+---------+--------------+-------------------+---------------------------+--------------+--------------------+------------------------+
| Año|     Familia|                Tipo|Compuesto|BajasEmisiones|         ValorAnual|PorcentajeAnualSobreFamilia| TotalGenerado|PorcentajeSobreTotal|PorcentajeBajasEmisiones|
+----+------------+--------------------+---------+--------------+-------------------+---------------------------+--------------+--------------------+------------------------+
|2016|No-Renovable|    Turbina de vapor|    false|         false| 2536143.0030000005|         0.0167077641039327|2.5700810617E8|                0.99|                   61.13|
|2016|No-Renovable|      Turbina de gas|    false|         false|  616037.3019999998|       0.003893066689377...|2.5700810617E8|                0.24|                   61.13|
|2016|   Renovable|       Solar térmica|    false|          true|  5071201.701999999|       0.054412720483706205|2.5700810617

startCsv: Long = 704682062526900L
ventanaCsv: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@503e0a9d
datosBalanceAnualesCsv: DataFrame = [Año: string, Familia: string ... 8 more fields]
endCsv: Long = 704682875117900L

In [19]:
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.functions._

// 📌 Leer datos desde CSV con inferSchema activado
val pathCsv = "../../data/csv/dsBalanceNacional11-25.csv"

val startCsv = System.nanoTime()

val dfFromCsv = spark.read
    .option("header", "true")
    .option("inferSchema", "true") // Spark inferirá automáticamente los tipos de datos
    .csv(pathCsv)

val ventanaCsv = Window.partitionBy("Año")

val datosBalanceAnualesCsv = dfFromCsv
    .withColumn("Año", date_format($"Fecha", "yyyy"))
    .groupBy($"Año", $"Familia", $"Tipo", $"Compuesto", $"BajasEmisiones")
    .agg(
        sum("Valor").as("ValorAnual"),
        avg("Porcentaje").as("PorcentajeAnualSobreFamilia")
    )
    .withColumn(
        "TotalGenerado", 
        round(
            sum(
                when(!$"Compuesto" && $"Familia" =!= "Demanda", $"ValorAnual")
                    .otherwise(0)).over(ventanaCsv),2)
    )
    .withColumn(
        "PorcentajeSobreTotal", 
        round($"ValorAnual" / $"TotalGenerado" * 100,2))
    .withColumn(
        "PorcentajeBajasEmisiones", 
        round(
            sum(
                when($"BajasEmisiones" && !$"Compuesto" && $"Familia" =!= "Demanda", $"ValorAnual")
                    .otherwise(0)).over(ventanaCsv) / $"TotalGenerado" * 100,2)
    )

// Mostrar resultados y calcular tiempo de ejecución
datosBalanceAnualesCsv.show(5)

val endCsv = System.nanoTime()
println(s"Tiempo ejecución CSV: ${(endCsv - startCsv) / 1e9} segundos")


+----+------------+--------------------+---------+--------------+-------------------+---------------------------+--------------+--------------------+------------------------+
| Año|     Familia|                Tipo|Compuesto|BajasEmisiones|         ValorAnual|PorcentajeAnualSobreFamilia| TotalGenerado|PorcentajeSobreTotal|PorcentajeBajasEmisiones|
+----+------------+--------------------+---------+--------------+-------------------+---------------------------+--------------+--------------------+------------------------+
|2016|No-Renovable|    Turbina de vapor|    false|         false| 2536143.0030000005|         0.0167077641039327|2.5700810617E8|                0.99|                   61.13|
|2016|No-Renovable|      Turbina de gas|    false|         false|  616037.3019999998|       0.003893066689377...|2.5700810617E8|                0.24|                   61.13|
|2016|   Renovable|       Solar térmica|    false|          true|  5071201.701999999|       0.054412720483706205|2.5700810617

import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.functions._
pathCsv: String = "../../data/csv/dsBalanceNacional11-25.csv"
startCsv: Long = 705252237406700L
dfFromCsv: DataFrame = [Familia: string, Tipo: string ... 5 more fields]
ventanaCsv: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@6959bb9e
datosBalanceAnualesCsv: DataFrame = [Año: string, Familia: string ... 8 more fields]
endCsv: Long = 705253387328300L

In [20]:
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.functions._

// 📌 Leer datos desde Parquet
val pathParquet = "../../data/parquet/dsBalanceNacional11-25.parquet"

val startParquet = System.nanoTime()

val dfFromParquet = spark.read
    .parquet(pathParquet)

val ventanaParquet = Window.partitionBy("Año")

val datosBalanceAnualesParquet = dfFromParquet
    .withColumn("Año", date_format($"Fecha", "yyyy"))
    .groupBy($"Año", $"Familia", $"Tipo", $"Compuesto", $"BajasEmisiones")
    .agg(
        sum("Valor").as("ValorAnual"),
        avg("Porcentaje").as("PorcentajeAnualSobreFamilia")
    )
    .withColumn(
        "TotalGenerado", 
        round(
            sum(
                when(!$"Compuesto" && $"Familia" =!= "Demanda", $"ValorAnual")
                    .otherwise(0)).over(ventanaParquet),2)
    )
    .withColumn(
        "PorcentajeSobreTotal", 
        round($"ValorAnual" / $"TotalGenerado" * 100,2))
    .withColumn(
        "PorcentajeBajasEmisiones", 
        round(
            sum(
                when($"BajasEmisiones" && !$"Compuesto" && $"Familia" =!= "Demanda", $"ValorAnual")
                    .otherwise(0)).over(ventanaParquet) / $"TotalGenerado" * 100,2)
    )

// Mostrar resultados y calcular tiempo de ejecución
datosBalanceAnualesParquet.show(5)

val endParquet = System.nanoTime()
println(s"Tiempo ejecución Parquet: ${(endParquet - startParquet) / 1e9} segundos")


+----+------------+--------------------+---------+--------------+-------------------+---------------------------+--------------+--------------------+------------------------+
| Año|     Familia|                Tipo|Compuesto|BajasEmisiones|         ValorAnual|PorcentajeAnualSobreFamilia| TotalGenerado|PorcentajeSobreTotal|PorcentajeBajasEmisiones|
+----+------------+--------------------+---------+--------------+-------------------+---------------------------+--------------+--------------------+------------------------+
|2016|No-Renovable|    Turbina de vapor|    false|         false| 2536143.0030000005|         0.0167077641039327|2.5700810617E8|                0.99|                   61.13|
|2016|No-Renovable|      Turbina de gas|    false|         false|  616037.3019999998|       0.003893066689377...|2.5700810617E8|                0.24|                   61.13|
|2016|   Renovable|       Solar térmica|    false|          true|  5071201.701999999|       0.054412720483706205|2.5700810617

import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.functions._
pathParquet: String = "../../data/parquet/dsBalanceNacional11-25.parquet"
startParquet: Long = 705273071622100L
dfFromParquet: DataFrame = [Familia: string, Tipo: string ... 5 more fields]
ventanaParquet: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@3c562df2
datosBalanceAnualesParquet: DataFrame = [Año: string, Familia: string ... 8 more fields]
endParquet: Long = 705273878471200L